# Part 0 Introduction

9 February 2024  
This is the notebook used for the calculations underlying the frameworks comparison paper. Within we undertake the following steps:
- Import the locally held version of the CACTUS package (import name is as workbench).
- Initialise the shell of a CACTUS project using the template Rhino/Grasshopper files which we then used to build the basic BIPV facade and Radiance project folder.
- 

## Part 0.1 Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:

import sys
import subprocess
import os
import pandas as np
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pathlib
import re
import glob
import pandas as pd
import pyarrow.feather as feather

from pprint import pprint
module_path = r"C:\Users\Justin\Documents\GitHub\IPV_Workbench"
sys.path.insert(0, module_path)
import workbench
from workbench.utilities import general, config_utils, temporal, io
from workbench.manage import manage, host
# from workbench.manage import host
from workbench.workflows import workflows
from workbench.simulations import method_iv_solver, method_topology_solver
from workbench.visualize import plots as ipv_plot

# System Analysis

## 2.1 Irradiance

In [2]:
# because we have restarted our notebook we will reactivate our project by reading in the config path
# place your config path here
config_path = r"C:\Users\Justin\Desktop\cactus_framework_study.config"
project_manager = manage.Project(config_path)

# rerun the setup in order to rebuild the entire project object and its attributes
project_manager.project_setup()

# set the analysis period to 1 hour of the year, it does not matter which hour
project_manager.edit_cfg_file("analysis", "analysis_period", "10-11")

# load building
building = host.Host(project_manager)


Destination file already exists, copy aborted./nC:\Users\Justin\Nextcloud\Projects\17_framework\cactus_framework_study\inputs\shared\tmy\base.epw
Destination file already exists, copy aborted./nC:\Users\Justin\Nextcloud\Projects\17_framework\cactus_framework_study\inputs\hosts\B1111\base\geometry\0_cactus_geometry_template.3dm
Destination file already exists, copy aborted./nC:\Users\Justin\Nextcloud\Projects\17_framework\cactus_framework_study\inputs\hosts\B1111\base\geometry\0_cactus_geometry_template.gh
Destination file already exists, copy aborted./nC:\Users\Justin\Nextcloud\Projects\17_framework\cactus_framework_study\inputs\hosts\B1111\base\geometry\1_cactus_panelizer_template.gh
The project is initialized. We have created a base host object named 'B1111'.
You will need to either move or create the geometry and panelizer files into the appropriate directories.
The geometry files should follow the convention defined in the output of the template grasshopper and rhino files.
The pan

In [4]:
# because we will rewrite the irradiance grid with a set value each time we use it the accuracy can be greatly reduced to improve sim time
project_manager.edit_cfg_file("irradiance", "radiance_param_rflux", "-lw 1 -ab 1 -ad 500")
project_manager.edit_cfg_file("irradiance", "radiance_param_rcontrib", "-ad 128 -lw 1.0e-1 -dc 1 -dt 0 -dj 0")


In [5]:
# run irradiance simulation to create the structured files needed
workflows.run_irradiance(building, overwrite=True)

Starting Radiance workflow for surface 1111_0.
 - Running 2-Phase DDS with 1 workers
 - Current surface is 1111_0
 - Initializing the weather file.
 - Starting Part 1 (total).
     - oconv
     - rfluxmtx
         - Using Accelerad.
     - gendaymtx
     - dctimestep | rmtxop
 - Starting Part 2 (direct).
     - oconv
     - rfluxmtx
         - Using Accelerad.
     - gendaymtx
     - dctimestep | rmtxop
 - Starting Part 3 (sun).
     - create_primitive_sun
     - rcalc
     - oconv
     - rcontrib
         - Using Accelerad.
     - gendaymtx
     - dctimestep | rmtxop
Completed in 577.52 seconds.
Converting .ill files to feather for surface 1111_0.
 - Saving Irradiance results
    - Direct sensor data saved in compressed format, time=0.0-seconds.
    - Diffuse sensor data saved in compressed format, time=0.0-seconds.
Completed in 17.34 seconds.
-----------------------


In [3]:
building.get_cells_xyz("{1111;0}","{1111;0;83}")

array([[ 0.5375, 10.    ,  0.0925],
       [ 0.6875, 10.    ,  0.0925],
       [ 0.8375, 10.    ,  0.0925],
       [ 0.9875, 10.    ,  0.0925],
       [ 1.1375, 10.    ,  0.0925],
       [ 1.2875, 10.    ,  0.0925],
       [ 1.4375, 10.    ,  0.0925],
       [ 1.5875, 10.    ,  0.0925],
       [ 1.7375, 10.    ,  0.0925],
       [ 1.8875, 10.    ,  0.0925],
       [ 0.5375, 10.    ,  0.2425],
       [ 0.6875, 10.    ,  0.2425],
       [ 0.8375, 10.    ,  0.2425],
       [ 0.9875, 10.    ,  0.2425],
       [ 1.1375, 10.    ,  0.2425],
       [ 1.2875, 10.    ,  0.2425],
       [ 1.4375, 10.    ,  0.2425],
       [ 1.5875, 10.    ,  0.2425],
       [ 1.7375, 10.    ,  0.2425],
       [ 1.8875, 10.    ,  0.2425],
       [ 0.5375, 10.    ,  0.3925],
       [ 0.6875, 10.    ,  0.3925],
       [ 0.8375, 10.    ,  0.3925],
       [ 0.9875, 10.    ,  0.3925],
       [ 1.1375, 10.    ,  0.3925],
       [ 1.2875, 10.    ,  0.3925],
       [ 1.4375, 10.    ,  0.3925],
       [ 1.5875, 10.    ,  0

In [4]:
small_irradiance_results = io.load_irradiance_file(building.project, "1111_0", "diffuse")

In [6]:
# build new irradiance results files 
n_cols = io.load_grid_file(building.project, "1111_0")['X'].unique().shape[0]
n_rows = io.load_grid_file(building.project, "1111_0")['Z'].unique().shape[0]

full_irrad = np.zeros((n_rows,n_cols)) + 1000
full_irrad.shape

(275, 764)

In [7]:
idx = small_irradiance_results.index

df = pd.DataFrame(np.array([full_irrad.flatten()]*len(idx)))
df.set_index(idx,inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,210090,210091,210092,210093,210094,210095,210096,210097,210098,210099
2030-01-01 00:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2030-01-01 01:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2030-01-01 02:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2030-01-01 03:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2030-01-01 04:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-12-31 19:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2030-12-31 20:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2030-12-31 21:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2030-12-31 22:30:00,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0


In [8]:
feather.write_feather(df, r"C:\Users\Justin\Nextcloud\Projects\17_framework\notebooks\synthetic_irradiance.lz4", compression='lz4')